In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# 1. Import packages
import os, glob
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input

# 2. Paths and constants
IMG_DIR    = "/content/drive/MyDrive/feature"            # directory of images to visualize
OUT_DIR    = "/content/drive/MyDrive/feature_cam"        # output directory
MODEL_PATH = "/content/drive/MyDrive/resnet50_model_1.h5" # trained ResNet50 model path
IMG_SIZE   = (224, 224)
os.makedirs(OUT_DIR, exist_ok=True)

# 3. Load the trained model (Sequential)
base_model = load_model(MODEL_PATH, compile=False)
backbone = base_model.get_layer("resnet50")  # name as shown in model summary
idx = base_model.layers.index(backbone)

# 4. Rebuild the forward graph in Functional API
inp = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = inp
x = backbone(x, training=False)            # pass through ResNet50
conv_feat_tensor = x                       # convolution features before GAP (7x7x2048)

# Attach layers after the backbone from the original model (GAP, Dropout, Dense, etc.)
x_tmp = x
for layer in base_model.layers[idx + 1:]:
    x_tmp = layer(x_tmp)

func_model = Model(inputs=inp, outputs=x_tmp, name="functional_rebuilt")
print("Functional model rebuilt.")

# 5. Build grad_model: input -> [last conv features, final prediction]
grad_model = Model(inputs=func_model.input,
                   outputs=[conv_feat_tensor, func_model.output])

def make_gradcam_heatmap(img_array, use_logit=False):
    """Compute Grad-CAM heatmap and return (heatmap, probability_of_AI)."""
    with tf.GradientTape() as tape:
        conv_out, preds = grad_model(img_array, training=False)  # conv_out: (1,H,W,C)
        p = preds[:, 0]  # Dense(1, sigmoid) -> probability of AI
        if use_logit:
            eps = 1e-7
            p = tf.clip_by_value(p, eps, 1 - eps)
            target = tf.math.log(p / (1 - p))
        else:
            target = p
    grads = tape.gradient(target, conv_out)
    pooled = tf.reduce_mean(grads, axis=(0, 1, 2))    # (C,)
    conv_map = conv_out[0]                             # (H, W, C)
    heatmap = tf.reduce_sum(conv_map * pooled, axis=-1)
    heatmap = tf.nn.relu(heatmap)
    heatmap = heatmap / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy(), float(p.numpy()[0])

def overlay_heatmap(heatmap, bgr_img, alpha=0.45):
    """Overlay a heatmap on a BGR image."""
    h, w = bgr_img.shape[:2]
    heatmap = cv2.resize(heatmap, (w, h))
    heatmap_uint8 = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
    return cv2.addWeighted(heatmap_color, alpha, bgr_img, 1 - alpha, 0)



Functional model rebuilt.


In [3]:
# 6. Batch processing: display and save results
label_map = {0: "Human", 1: "AI"}
allow_ext = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
img_paths = sorted(glob.glob(os.path.join(IMG_DIR, "*.*")))

processed = 0
for p in img_paths:
    if os.path.splitext(p)[1].lower() not in allow_ext:
        continue

    bgr = cv2.imread(p)
    if bgr is None:
        print(f"[Skip] Cannot read: {p}")
        continue

    bgr_resized = cv2.resize(bgr, IMG_SIZE)

    ximg = image.img_to_array(cv2.cvtColor(bgr_resized, cv2.COLOR_BGR2RGB))
    ximg = np.expand_dims(ximg, axis=0)
    ximg = preprocess_input(ximg)  # ResNet50 preprocessing

    heatmap, prob_ai = make_gradcam_heatmap(ximg, use_logit=False)
    pred_cls = 1 if prob_ai >= 0.5 else 0

    cam_bgr = overlay_heatmap(heatmap, bgr_resized, alpha=0.45)

    # Prepare for display
    rgb_orig = cv2.cvtColor(bgr_resized, cv2.COLOR_BGR2RGB)
    rgb_cam  = cv2.cvtColor(cam_bgr,     cv2.COLOR_BGR2RGB)

    # Plot: show and save
    plt.figure(figsize=(6.5, 3.2), dpi=200)
    plt.subplot(1, 2, 1); plt.imshow(rgb_orig); plt.axis("off"); plt.title("Original", fontsize=9)
    plt.subplot(1, 2, 2); plt.imshow(rgb_cam);  plt.axis("off")
    plt.title(f"Grad-CAM - Pred: {label_map[pred_cls]} (AI prob = {prob_ai:.2f})", fontsize=9)
    plt.tight_layout()

    out_name = os.path.splitext(os.path.basename(p))[0] + "_cam.jpg"
    out_path = os.path.join(OUT_DIR, out_name)
    plt.savefig(out_path, bbox_inches="tight")
    plt.show()  # display inline
    plt.close()

    processed += 1
    print(f"[Saved] {out_path}")

print(f"Done. Exported {processed} Grad-CAM images to {OUT_DIR}.")


Output hidden; open in https://colab.research.google.com to view.

In [4]:
base_model.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)